In [919]:
from tools import *

In [921]:
import import_ipynb
from Model_4 import *

### Model 5a: Minimize Inbound and Outbound Logistics Constrained on Customer's Demand and Factory Capacity and Sufficient Supply (Multiple Ports) (Lossless Production) (Multiple Products) (Identical Production)

#### Input to the Model: 

1. Number of factories and customers and ports and products
2. Outbound cost (times the number of products)
3. Inbound cost (each port to each factory) (times the number of products)
4. Demands (times the number of products)
5. Factory Capacity (Identical)
6. Loss Percentage / Efficiency (None)

In [1153]:
no_I, no_J, no_K, no_P = 4, 5, 3, 2 # Number of factories, number of customers, number of ports

ibik = np.random.rand(no_P, no_I, no_K) # Inbound Cost
obij = np.random.rand(no_P, no_I, no_J) # Outbound Cost
wj = np.random.rand(no_P, no_J) # Demands vector
ki = np.random.rand(no_I) # Factory Capacity
#ei = np.random.rand(no_I) # Efficiency
ei = np.ones((no_P, no_I)) # Perfect Efficiency

while np.sum(wj) >= np.sum(ki):
    wj = np.random.rand(no_P, no_J) # Redo Demands vector
    ki = np.random.rand(no_I) # Redo Factory Capacity

In [1174]:
def generate_objective_vector(no_I, no_J, no_K, no_P, ibik, obij):
    c = np.concatenate([ibik.reshape(no_P, no_I, no_K), 
                        obij.reshape(no_P, no_I, no_J)], axis = 2).flatten()
    return c

In [1175]:
from scipy.linalg import block_diag

I have a feeling that this matrix will look exactly like |I| * |P| factories, no it's not you gotta create the base matrix and shift it over. so like block diag stuff

In [1176]:
def generate_demand_matrix(no_I, no_J, no_K, no_P):
    
    sub_block = np.tile(np.hstack([np.zeros((no_J, no_K)), np.eye(no_J)]), reps = no_I)
    
    Wijk = block_diag(*[sub_block]*no_P)
    
    return Wijk

In [1079]:
0 0 0 0| 1 1 1 1 1 1 | 0 0 0 0 | 0 0 0 0 0 0 || 0 0 0 0 | 1 1 1 1 1 1 | 0 0 0 0 | 0 0 0 0 0 0
0 0 0 0| 0 0 0 0 0 0 | 0 0 0 0 | 1 1 1 1 1 1 || 0 0 0 0 | 0 0 0 0 0 0 | 0 0 0 0 | 1 1 1 1 1 1

SyntaxError: invalid syntax (<ipython-input-1079-f2e9395a5b41>, line 1)

It looks like the capacity matrix is just like before but now you tile the individual matrix |P| times

In [1177]:
def generate_capacity_matrix(no_I, no_J, no_K, no_P):
    
    factory_block_list = []

    port_block = np.zeros((no_I, no_K))

    for i in range(no_I):
        customer_column = np.zeros((no_I, 1))
        customer_column[i] = 1
        customer_block = np.repeat(customer_column, repeats = no_J, axis = 1)
        factory_block_list.append(np.hstack([port_block, customer_block]))

    Ki = np.tile(np.concatenate(factory_block_list, axis = 1), reps = no_P)
    
    return Ki

In [1178]:
## Assume non-trivial

Wijk = generate_demand_matrix(no_I, no_J, no_K, no_P) # Demand Constraint Matrix
Ki = generate_capacity_matrix(no_I, no_J, no_K, no_P) # Capacity Constraint Matrix

In [1208]:
Matrix(Si)

Matrix([
[-1.0, -1.0, -1.0, 1.0, 1.0, 1.0, 1.0, 1.0,  0.0,  0.0,  0.0, 0.0, 0.0, 0.0, 0.0, 0.0,  0.0,  0.0,  0.0, 0.0, 0.0, 0.0, 0.0, 0.0,  0.0,  0.0,  0.0, 0.0, 0.0, 0.0, 0.0, 0.0,  0.0,  0.0,  0.0, 0.0, 0.0, 0.0, 0.0, 0.0,  0.0,  0.0,  0.0, 0.0, 0.0, 0.0, 0.0, 0.0,  0.0,  0.0,  0.0, 0.0, 0.0, 0.0, 0.0, 0.0,  0.0,  0.0,  0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
[ 0.0,  0.0,  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1.0, -1.0, -1.0, 1.0, 1.0, 1.0, 1.0, 1.0,  0.0,  0.0,  0.0, 0.0, 0.0, 0.0, 0.0, 0.0,  0.0,  0.0,  0.0, 0.0, 0.0, 0.0, 0.0, 0.0,  0.0,  0.0,  0.0, 0.0, 0.0, 0.0, 0.0, 0.0,  0.0,  0.0,  0.0, 0.0, 0.0, 0.0, 0.0, 0.0,  0.0,  0.0,  0.0, 0.0, 0.0, 0.0, 0.0, 0.0,  0.0,  0.0,  0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
[ 0.0,  0.0,  0.0, 0.0, 0.0, 0.0, 0.0, 0.0,  0.0,  0.0,  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1.0, -1.0, -1.0, 1.0, 1.0, 1.0, 1.0, 1.0,  0.0,  0.0,  0.0, 0.0, 0.0, 0.0, 0.0, 0.0,  0.0,  0.0,  0.0, 0.0, 0.0, 0.0, 0.0, 0.0,  0.0,  0.0,  0.0, 0.0, 0.0, 0.0, 0.0, 0.0,  0.0,  0.0,  0.0, 0.0, 0.0, 0.0, 0.0, 0.

In [1227]:
def generate_supply_matrix(no_I, no_J, no_K, no_P):

    sub_block = [-1]*no_K + [1]*no_J
    
    Si = block_diag(*[sub_block]*no_I*no_P)
    
    return Si

In [1228]:
## Standard form of our model

# New cost vector
c = generate_objective_vector(no_I, no_J, no_K, no_P, ibik, obij)

# Upper Bound
A_ub = np.vstack([-Wijk, Ki, Si])
b_ub = np.hstack([-wj.flatten(), ki.flatten(), np.zeros(no_I*no_P)])

prog_ub = linprog(c, A_ub = A_ub, b_ub = b_ub) # Bigger than or equal constraints

In [1229]:
prog_ub

     con: array([], dtype=float64)
     fun: 1.5054664047301745
 message: 'Optimization terminated successfully.'
     nit: 9
   slack: array([ 4.21034891e-11,  2.89366864e-11,  2.68947087e-11,  3.78488629e-11,
        3.16659476e-11,  2.55011567e-11,  2.43177700e-11,  3.06328990e-11,
        3.12677662e-11,  2.85765300e-11, -7.19086457e-11,  1.44522929e-02,
       -5.12192511e-11, -7.18975990e-11,  1.07934994e-10,  8.77900050e-11,
       -1.32283192e-11,  7.46705919e-10, -1.46698695e-11, -1.60743988e-11,
       -1.00907060e-11,  2.60513833e-12])
  status: 0
 success: True
       x: array([3.00558566e-01, 3.31446102e-12, 1.38071569e-11, 8.66027330e-03,
       4.65090189e-12, 2.91898292e-01, 5.35277802e-12, 5.20685676e-12,
       4.23863685e-01, 1.70942718e-11, 4.19173136e-12, 3.84391404e-11,
       4.23863685e-01, 1.25654109e-10, 5.34945502e-12, 1.07401869e-11,
       7.62061543e-12, 4.79258448e-12, 2.72031571e-10, 3.34572369e-12,
       7.24414417e-11, 1.93308080e-10, 2.20973062e-11, 

In [1230]:
result = prog_ub.x.reshape(no_P, no_I, no_J+no_K)

In [1231]:
Matrix(result[0])

Matrix([
[   0.300558565628107, 3.31446102428792e-12, 1.38071568894722e-11,  0.00866027330040877,  4.6509018943007e-12,    0.291898292221674,  5.3527780215862e-12, 5.20685675956531e-12],
[   0.423863685013507, 1.70942717562551e-11, 4.19173135926597e-12, 3.84391403951562e-11,     0.42386368476682, 1.25654108923904e-10, 5.34945501889596e-12, 1.07401869386202e-11],
[7.62061542989802e-12, 4.79258448444286e-12, 2.72031571016308e-10, 3.34572369238719e-12, 7.24414417170596e-11, 1.93308080403422e-10, 2.20973061594738e-11,  6.4805381526451e-12],
[ 1.5660464555302e-11,    0.278263029511988, 3.12043343073827e-12, 1.31273604348229e-11,  1.8162508402345e-11,  3.84298897484762e-9,    0.157183156453459,    0.121079868456326]])

In [1232]:
Matrix(result[1])

Matrix([
[8.09475483357298e-12, 1.23518058934616e-11,   0.0507253866349887, 1.70939496729186e-11, 1.75860608169655e-11, 1.60422707778048e-10, 3.58857040338395e-12,   0.0507253864714139],
[ 4.8636806345216e-12, 4.31887516963436e-12,    0.133323681282349, 6.81172225523418e-11, 1.34584346664612e-10,   0.0318508660060446,    0.101472815092119, 6.74067850975834e-12],
[2.24215093528941e-11,    0.969915391626939, 5.68821006857345e-12,    0.397743497773202,    0.572171893857018, 7.08967207189572e-12, 4.10562909156495e-12, 2.37235488769312e-11],
[   0.449602552799513,  3.8410293338578e-12, 4.77906870462582e-12,    0.160130778757358, 1.51408845692814e-11,  7.3119050121529e-12,      0.1680533160287,    0.121418457997017]])

In [1233]:
Matrix(result[:, :, no_K:][0])

Matrix([
[ 0.00866027330040877,  4.6509018943007e-12,    0.291898292221674,  5.3527780215862e-12, 5.20685675956531e-12],
[3.84391403951562e-11,     0.42386368476682, 1.25654108923904e-10, 5.34945501889596e-12, 1.07401869386202e-11],
[3.34572369238719e-12, 7.24414417170596e-11, 1.93308080403422e-10, 2.20973061594738e-11,  6.4805381526451e-12],
[1.31273604348229e-11,  1.8162508402345e-11,  3.84298897484762e-9,    0.157183156453459,    0.121079868456326]])

In [1234]:
### Check demand, note that the axis = 1 the array is 3d
aae(np.sum(result[:, :, no_K:], axis = 1), wj)

In [1235]:
### Check weighted inbound volume vs outbound volume
aae(np.sum(result[:, :, :no_K], axis = 2), np.sum(result[:, :, no_K:], axis = 2))

#### Modular Code

In [1281]:
def optimize_logistics_5a(no_I, no_J, no_K, ibik, obij, wj, ki):
    
    assert np.sum(ki) >= np.sum(wj), 'More Demand than Capacity. Program is Impossible'
    
    ## Assume non-trivial
    Wijk = generate_demand_matrix(no_I, no_J, no_K, no_P) # Demand Constraint Matrix
    Ki = generate_capacity_matrix(no_I, no_J, no_K, no_P) # Capacity Constraint Matrix
    
    # Sufficient Supply Matrix
    Si = generate_supply_matrix(no_I, no_J, no_K, no_P)

    ## Standard form of our model

    # New cost vector
    c = generate_objective_vector(no_I, no_J, no_K, no_P, ibik, obij)

    # Upper Bound
    A_ub = np.vstack([-Wijk, Ki, Si])
    b_ub = np.hstack([-wj.flatten(), ki.flatten(), np.zeros(no_I*no_P)])

    prog = linprog(c, A_ub = A_ub, b_ub = b_ub) # Bigger than or equal constraints
    
    result = prog.x.reshape(no_P, no_I, no_J+no_K)
    
    ### Check demand, note that the axis = 1 the array is 3d
    aae(np.sum(result[:, :, no_K:], axis = 1), wj)
    
    ### Check weighted inbound volume vs outbound volume
    aae(np.sum(result[:, :, :no_K], axis = 2), np.sum(result[:, :, no_K:], axis = 2))
    
    return prog

### Model 5b: Minimize Inbound and Outbound Logistics Constrained on Customer's Demand and Factory Capacity and Sufficient Supply (Multiple Ports) (Lossy Production) (Multiple Products) (Identical Production)

We are going to skip the individual factory / product efficiency and go for full, but by the end, we will also talk about how to do individual case

#### Input to the Model: 

1. Number of factories and customers and ports and products
2. Outbound cost (times the number of products)
3. Inbound cost (each port to each factory) (times the number of products)
4. Demands (times the number of products)
5. Factory Capacity (Identical)
6. Loss Percentage / Efficiency (Full)

In [1307]:
no_I, no_J, no_K, no_P = 2, 10, 1, 1 # Number of factories, number of customers, number of ports

ibik = np.random.rand(no_P, no_I, no_K) # Inbound Cost
obij = np.random.rand(no_P, no_I, no_J) # Outbound Cost
wj = np.random.rand(no_P, no_J) # Demands vector
ki = np.random.rand(no_I) # Factory Capacity
ei = np.random.rand(no_P, no_I) # Efficiency If by-product: row same, by-factory: column same\

while np.sum(wj) >= np.sum(ki):
    wj = np.random.rand(no_P, no_J) # Redo Demands vector
    ki = np.random.rand(no_I) # Redo Factory Capacity

This only changes the sufficient supply matrix

Hmmm. Not entirely obvious how to do this. Let's take a look at the previous example

In [1298]:
Matrix(Si)

Matrix([
[-0.147023171162653, -0.147023171162653, -0.147023171162653, 1.0, 1.0, 1.0, 1.0, 1.0,                0.0,                0.0,                0.0, 0.0, 0.0, 0.0, 0.0, 0.0,                0.0,                0.0,                0.0, 0.0, 0.0, 0.0, 0.0, 0.0,                0.0,                0.0,                0.0, 0.0, 0.0, 0.0, 0.0, 0.0,                0.0,                0.0,                0.0, 0.0, 0.0, 0.0, 0.0, 0.0,                0.0,                0.0,                0.0, 0.0, 0.0, 0.0, 0.0, 0.0,                0.0,                0.0,                0.0, 0.0, 0.0, 0.0, 0.0, 0.0,                0.0,                0.0,                0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
[               0.0,                0.0,                0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.943579576677984, -0.943579576677984, -0.943579576677984, 1.0, 1.0, 1.0, 1.0, 1.0,                0.0,                0.0,                0.0, 0.0, 0.0, 0.0, 0.0, 0.0,                0.0,                0.0,              

In [1299]:
def generate_supply_matrix_with_efficiency(no_I, no_J, no_K, no_P, ei):

    left = np.tile(-ei.reshape(no_P*no_I, 1), reps = no_K)

    right = np.ones((no_P*no_I, no_J))

    Si = block_diag(*np.hstack([left, right]))
    
    return Si

In [1308]:
def optimize_logistics_5b(no_I, no_J, no_K, ibik, obij, wj, ki, ei):
    
    assert np.sum(ki) >= np.sum(wj), 'More Demand than Capacity. Program is Impossible'
    
    ## Assume non-trivial
    Wijk = generate_demand_matrix(no_I, no_J, no_K, no_P) # Demand Constraint Matrix
    Ki = generate_capacity_matrix(no_I, no_J, no_K, no_P) # Capacity Constraint Matrix
    
    # Sufficient Supply Matrix
    Si = generate_supply_matrix_with_efficiency(no_I, no_J, no_K, no_P, ei)

    ## Standard form of our model

    # New cost vector
    c = generate_objective_vector(no_I, no_J, no_K, no_P, ibik, obij)

    # Upper Bound
    A_ub = np.vstack([-Wijk, Ki, Si])
    b_ub = np.hstack([-wj.flatten(), ki.flatten(), np.zeros(no_I*no_P)])

    prog = linprog(c, A_ub = A_ub, b_ub = b_ub) # Bigger than or equal constraints
    
    result = prog.x.reshape(no_P, no_I, no_J+no_K)
    
    ### Check demand, note that the axis = 1 the array is 3d
    aae(np.sum(result[:, :, no_K:], axis = 1), wj)
    
    ### Check weighted inbound volume vs outbound volume
    aae(np.sum(result[:, :, :no_K], axis = 2)*ei, np.sum(result[:, :, no_K:], axis = 2))
    
    return prog

In [1309]:
prog = optimize_logistics_5b(no_I, no_J, no_K, ibik, obij, wj, ki, ei)